In [ ]:
# Celdas para leer directamente un archivo .arff en Python

# 1. Importar librerías necesarias
import pandas as pd
from scipy.io import arff
# Removed incorrect import of load_arff
from pathlib import Path
from sklearn.preprocessing import LabelEncoder

: 

In [ ]:
# 2. Definir la ruta al archivo .arff
darff_path = Path('Sapfile1.arff')  # <-- Ajusta si está en otra carpeta

# 3. Leer el .arff usando scipy
#    Devuelve una tupla (data, meta)
data_scipy, meta_scipy = arff.loadarff(str(darff_path))
# Convertir a DataFrame
df_scipy = pd.DataFrame(data_scipy)
# Decodificar columnas byte a texto si es necesario
for col in df_scipy.select_dtypes([object]):
    df_scipy[col] = df_scipy[col].str.decode('utf-8')
print("Primeras filas con scipy:")
print(df_scipy.head())

# Removed scikit-learn related code as load_arff is not available

# Opcional: mostrar información general
def resumen(df):
    print(df.info())
    print(df.describe())

# resumen(df_scipy)


In [ ]:
# Verificar cantidad de filas (instancias)
print("Número de instancias:", df_scipy.shape[0])

# Verificar valores nulos reales (NaN)
print("\nValores nulos por columna:")
print(df_scipy.isnull().sum())

# Buscar valores '?' como posibles datos faltantes
print("\nValores '?' por columna:")
for col in df_scipy.columns:
    if df_scipy[col].dtype == object:
        print(f"{col}: {(df_scipy[col] == '?').sum()}")


In [ ]:
print("Tipos de datos del DataFrame:")
print(df_scipy.dtypes)


In [ ]:
# Análisis de la variable objetivo
print("Valores únicos en 'atd':")
print(df_scipy['atd'].value_counts())


In [ ]:
# Llevar la columna 'atd' al final
target = 'atd'
df_scipy = df_scipy[[col for col in df_scipy.columns if col != target] + [target]]

# Crear copia para no tocar el original directamente
df_encoded = df_scipy.copy()

# Aplicar LabelEncoder a todas las columnas (incluyendo el target)
label_encoders = {}
for col in df_encoded.columns:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    label_encoders[col] = le  # guardar por si luego se necesita invertir

# Mostrar las primeras filas codificadas
df_encoded.head()